## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.65,115.33,48.99,86,39,9.28,scattered clouds
1,1,Bambous Virieux,MU,-20.34,57.76,69.80,77,40,8.05,scattered clouds
2,2,Port Elizabeth,ZA,-33.92,25.57,57.20,93,40,4.70,scattered clouds
3,3,Hilo,US,19.73,-155.09,86.00,66,1,10.29,clear sky
4,4,Barcelos,PT,41.54,-8.62,64.99,88,1,8.05,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you are looking for on your trip? "))
max_temp = float(input("What is the maximum temperature you are looking for on your trip? "))

What is the minimum temperature you are looking for on your trip? 65
What is the maximum temperature you are looking for on your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Bambous Virieux,MU,-20.34,57.76,69.80,77,40,8.05,scattered clouds
31,31,Norman Wells,CA,65.28,-126.83,68.00,52,40,13.87,scattered clouds
36,36,Rikitea,PF,-23.12,-134.97,67.01,73,87,18.52,overcast clouds
44,44,Svetlyy,RU,54.67,20.13,68.00,94,75,4.47,broken clouds
45,45,Senanga,ZM,-16.12,23.27,68.14,40,0,10.11,clear sky
48,48,Caravelas,BR,-17.71,-39.25,70.83,81,100,11.41,overcast clouds
58,58,Luanda,AO,-8.84,13.23,71.60,88,75,3.36,broken clouds
67,67,Am Timan,TD,11.03,20.28,72.39,95,100,1.66,overcast clouds
68,68,Fairbanks,US,64.84,-147.72,73.00,46,90,6.93,overcast clouds
86,86,Bousso,TD,10.48,16.71,71.91,92,100,2.93,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                147
City                   147
Country                145
Lat                    147
Lng                    147
Max Temp               147
Humidity               147
Cloudiness             147
Wind Speed             147
Current Description    147
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_final_df = preferred_cities_df.dropna()
preferred_cities_final_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Bambous Virieux,MU,-20.34,57.76,69.80,77,40,8.05,scattered clouds
31,31,Norman Wells,CA,65.28,-126.83,68.00,52,40,13.87,scattered clouds
36,36,Rikitea,PF,-23.12,-134.97,67.01,73,87,18.52,overcast clouds
44,44,Svetlyy,RU,54.67,20.13,68.00,94,75,4.47,broken clouds
45,45,Senanga,ZM,-16.12,23.27,68.14,40,0,10.11,clear sky
...,...,...,...,...,...,...,...,...,...,...
713,713,Calama,CL,-22.47,-68.93,66.20,20,0,19.46,clear sky
716,716,Takoradi,GH,4.88,-1.76,74.62,91,8,6.53,clear sky
717,717,Cap Malheureux,MU,-19.98,57.61,69.01,79,17,3.00,light rain
721,721,Tonantins,BR,-2.87,-67.80,74.68,70,0,2.17,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_final_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bambous Virieux,MU,69.80,scattered clouds,-20.34,57.76,
31,Norman Wells,CA,68.00,scattered clouds,65.28,-126.83,
36,Rikitea,PF,67.01,overcast clouds,-23.12,-134.97,
44,Svetlyy,RU,68.00,broken clouds,54.67,20.13,
45,Senanga,ZM,68.14,clear sky,-16.12,23.27,
48,Caravelas,BR,70.83,overcast clouds,-17.71,-39.25,
58,Luanda,AO,71.60,broken clouds,-8.84,13.23,
67,Am Timan,TD,72.39,overcast clouds,11.03,20.28,
68,Fairbanks,US,73.00,overcast clouds,64.84,-147.72,
86,Bousso,TD,71.91,overcast clouds,10.48,16.71,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.nan
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bambous Virieux,MU,69.80,scattered clouds,-20.34,57.76,Casa Tia Villa
31,Norman Wells,CA,68.00,scattered clouds,65.28,-126.83,Heritage Hotel
36,Rikitea,PF,67.01,overcast clouds,-23.12,-134.97,Pension Maro'i
44,Svetlyy,RU,68.00,broken clouds,54.67,20.13,"Gostinitsa-Kafe, Alena"
45,Senanga,ZM,68.14,clear sky,-16.12,23.27,Christopher Nasilele House


In [10]:
# 7. Drop the rows where there is no Hotel Name.
# 7a. Chacking for any null Hotel Names
hotel_df.count()

City                   145
Country                145
Max Temp               145
Current Description    145
Lat                    145
Lng                    145
Hotel Name             135
dtype: int64

In [11]:
# 7b. Removing Hotels with null values
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bambous Virieux,MU,69.80,scattered clouds,-20.34,57.76,Casa Tia Villa
31,Norman Wells,CA,68.00,scattered clouds,65.28,-126.83,Heritage Hotel
36,Rikitea,PF,67.01,overcast clouds,-23.12,-134.97,Pension Maro'i
44,Svetlyy,RU,68.00,broken clouds,54.67,20.13,"Gostinitsa-Kafe, Alena"
45,Senanga,ZM,68.14,clear sky,-16.12,23.27,Christopher Nasilele House
48,Caravelas,BR,70.83,overcast clouds,-17.71,-39.25,Pousada dos Navegantes
58,Luanda,AO,71.60,broken clouds,-8.84,13.23,EPIC SANA Luanda
68,Fairbanks,US,73.00,overcast clouds,64.84,-147.72,Westmark Fairbanks Hotel & Conference Center
96,Cassilandia,BR,66.15,scattered clouds,-19.11,-51.73,Hotel Rafa Cassilândia
99,Tamandare,BR,74.39,scattered clouds,-8.76,-35.10,Pousada 100 Passos


In [12]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))